In [1]:
import pandas as pd

import requests

import os

In [2]:
import tweepy

In [3]:
import geopy
from geopy.geocoders import Nominatim

In [4]:
auth = tweepy.AppAuthHandler('LkLnmiQ9tX1NLPdORsJVxoCEb','C32xJFK6i4EKECBfT31DFoF8xhyk2wxOjREA1DX588eWMfjHAE')

In [49]:
api = tweepy.API(auth)
output = []
for tweet in tweepy.Cursor(api.search, q='wildfires-filter:retweets',geocode='37.77986,-122.42905,2500km',until='2020-10-26').items(100):
    output.append({'date':tweet.created_at,'text':tweet.text,'geocode':tweet.place})
output = pd.DataFrame(output)

In [51]:
output.isnull().sum()

date        0
text        0
geocode    96
dtype: int64

In [52]:
final = pd.DataFrame(output)

In [53]:
final = pd.concat([final,output])

In [54]:
final

,date,text,geocode
0,2020-10-25 23:58:59,@Geno1955 Same here. I haven't shut my down on...,None
1,2020-10-25 23:57:08,@fakegingerbitch Ordinarily I would agree.\n\n...,None
2,2020-10-25 23:50:38,Shasta County has already seen three relativel...,None
3,2020-10-25 23:50:30,Colorado photo dump plus my car setup for my f...,None
4,2020-10-25 23:49:05,"@IdiosyncraticXL @Nightinghawk Dear Phil, I wi...",None
...,...,...,...
95,2020-10-25 20:07:18,@ContraPoints We’re about to evacuate some mor...,None
96,2020-10-25 20:03:02,Climate change is absolutely a Washington Stat...,None
97,2020-10-25 20:02:37,COVID and the wildfires here shook me more tha...,None
98,2020-10-25 20:00:41,California faces highest fire risk of 2020 wit...,None


In [89]:
import time

In [14]:
my_dates = ['2020-10-27','2020-10-26','2020-10-25','2020-10-24','2020-10-23','2020-10-22','2020-10-21','2020-10-20','2020-10-19']

In [5]:
def get_tweets(geocodee,date_range):
    api = tweepy.API(auth)
    output = []
    last = pd.DataFrame(columns=['date','text','geocode'])
    for date in date_range:
        for tweet in tweepy.Cursor(api.search, q='wildfires-filter:retweets',geocode=geocodee,until=date,tweet_mode='extended',lang='en').items(100):
            output.append({'date': tweet.created_at,'text':tweet.full_text,'geocode':tweet.place,'user_location':tweet.user.location})
        time.sleep(5)
    output = pd.DataFrame(output)
        
    return output

In [207]:
la_geo = '34.05349,-118.24532,200km'
sf_geo = '37.77986,-122.42905,300km'

In [198]:
la_4 = get_tweets(la_geo,my_dates)

In [203]:
la_4

,date,text,geocode,user_location
0,2020-10-26 17:40:11,LATEST: After first reporting that the #Silver...,None,"Los Angeles, CA"
1,2020-10-26 17:38:59,Planned power outages affect hundreds of thous...,None,"Altadena, CA"
2,2020-10-26 17:37:07,When writing political stories you often have ...,None,"Los Angeles, CA"
3,2020-10-26 17:36:15,"""Planned power outages affect hundreds of thou...",None,"Los Angeles, CA"
4,2020-10-26 17:32:54,If you are in a safe place after evacuating fr...,None,"Los Angeles, CA"
...,...,...,...,...
795,2020-10-18 22:51:06,Despite years of neglect and mismanagement in ...,None,"Los Angeles, CA"
796,2020-10-18 22:48:59,COVID - my gym has now been closed 31 weeks \n...,None,"Santa Clarita, CA"
797,2020-10-18 22:30:06,Wildfires and floods threaten tens of millions...,None,"San Diego, California"
798,2020-10-18 22:26:30,@lexis_dawson23 ask him about the type of worl...,None,Los Angeles


In [193]:
la_3F = get_tweets(la_geo,my_dates)

In [196]:
la_3F['text'][1]

'Planned power outages affect hundreds of thousands as intense wildfires rage on in Northern California https://t.co/zjs7Z5dMjf'

In [197]:
sf_3F = get_tweets(sf_geo,my_dates)

In [ ]:
sf_3F['text'][0]

In [205]:
best = pd.concat([la_3F,sf_3F])
best.reset_index(inplace=True,drop=True)

In [208]:
best.to_csv('twitter/best_tweets.csv',index=False)

In [209]:
best['user_location'].value_counts()

Los Angeles, CA              251
San Francisco, CA            181
San Diego, CA                110
Los Angeles                   64
Sacramento, CA                58
                            ... 
Claremont, CA                  1
Napa CA                        1
Danville, CA                   1
Los Angeles, CA, USA           1
Santa Cruz Mountains, USA      1
Name: user_location, Length: 334, dtype: int64

## Next Steps:

- Remove URL's from text.
- Get the geocodes from my User Location for plotting later.

## Problems Encountered:

- It took a lot of tinkering before I was able to search for wildfires in a specific range. To do this I needed to enter the latitidue,longitude,and range of kms or mi.
- Specifying I did not want my query to include retweets was also a necessary step.
- Many tweets did not include a Geotag which is why I pulled in User Location to be used as our geocode input later.
- We can only get tweets from the last 7 days. I would need to pay for Premium to get access to the entire database. I will continue to collect more tweets from smaller geo ranges and from today until Thursday. 
- Initially, tweets returned would contain only part of the tweet. I needed to specify in my query that I wanted extended tweets. After doing this, I was able to obtain full tweets.
- I created my function to make running through the entire date range more efficient. Initially, I was searching by individual date and needing to combine dataframes after each run. Very inefficient.


In [210]:
sd_geo = '32.71533,-117.15726,15mi'

In [211]:
sd_tweets = get_tweets(sd_geo,my_dates)

In [212]:
sd_tweets

,date,text,geocode,user_location
0,2020-10-26 17:58:23,Hundreds of thousands lose power as Northern C...,None,San Diego
1,2020-10-26 14:38:19,Two Colorado Wildfires Turn Deadly As Thousand...,None,"Chula Vista, CA"
2,2020-10-26 13:21:50,Large swaths of California were without power ...,None,"San Diego, CA"
3,2020-10-26 08:24:47,@Hey_its_Patti @SVNewsAlerts And the wildfires...,None,"San Diego, CA"
4,2020-10-26 03:50:02,With #California bracing for #FireSeason PG&am...,None,"San Diego, California"
...,...,...,...,...
606,2020-10-18 01:04:22,Wildfires have pushed Napa Valley’s wine indus...,None,"San Diego, CA"
607,2020-10-18 00:51:09,#TrumpTheSnake slithering into Orange County t...,None,"San Diego, CA"
608,2020-10-17 23:10:22,Bill Gates Wants To Dim The Sun And It Can Act...,None,by the Lake in Sunny San Diego
609,2020-10-17 22:50:02,Our friends @ATT offer a National Disaster R...,None,"San Diego, CA"


In [229]:
la_geo2 = '34.05349,-118.24532,10mi'

In [230]:
la_refined = get_tweets(la_geo2,my_dates)

In [231]:
la_refined

,date,text,geocode,user_location
0,2020-10-26 18:47:21,An abrupt change from cool fall weather to ext...,None,"Los Angeles, CA"
1,2020-10-26 18:32:29,Wildfires are looking really bad today through...,None,"Los Angeles, CA"
2,2020-10-26 18:30:20,We're definitely feeling the wind here in #NoH...,None,Los Angeles
3,2020-10-26 18:27:37,Why California's Devastating Wildfires Will Pu...,None,"Pasadena, Ca"
4,2020-10-26 18:27:10,UPDATE: Cal Fire has revised the number of acr...,None,"Los Angeles, CA"
...,...,...,...,...
769,2020-10-17 20:36:33,Wildfires and their impact on Skyline College ...,None,Los Angeles
770,2020-10-17 20:35:17,Trump’s quick reversal is consistent w/ his gu...,None,"West Hollywood, California"
771,2020-10-17 20:14:20,@diskettepark @malennoo @Skateboardp_xd @RipRj...,None,"Los Angeles, CA"
772,2020-10-17 19:33:52,"Black Dust (Acoustic) | Jameson Tabor, Taylor ...",None,"Los Angeles, CA"


In [232]:
santa_rosa_geo = '38.43808,-122.71247,10mi'

In [233]:
santa_rosa = get_tweets(santa_rosa_geo,my_dates)

In [234]:
santa_rosa

,date,text,geocode,user_location
0,2020-10-26 16:05:11,California fires live updates: Worst fire dang...,None,"Santa Rosa, CA"
1,2020-10-26 13:10:44,The agency says it will monitor the conditions...,None,"Santa Rosa, California"
2,2020-10-25 02:35:36,To all in the North Bay &amp; Bay Area\n\nYour...,None,"Santa Rosa, CA"
3,2020-10-24 17:00:43,"As we wrapped our October issue, the Glass Inc...",None,Santa Rosa CA
4,2020-10-24 01:12:41,I participated on a statewide panel on librari...,None,"Santa Rosa, CA"
...,...,...,...,...
114,2020-10-17 19:31:13,@choo_ek @5tephanieJean A critical point &amp;...,None,"Santa Rosa, CA"
115,2020-10-19 16:24:45,Cameron Peak Fire and other wildfires are prom...,None,"Santa Rosa, CA"
116,2020-10-19 02:56:06,@terri_____ Wine country and wildfires.,None,"Rohnert Park, CA"
117,2020-10-18 14:26:49,"Extended winter, widespread floods, raging wil...",None,"Santa Rosa, CA"


In [235]:
sf_geo2 = '37.77986,-122.42905,10mi'

In [236]:
sf_refined = get_tweets(sf_geo2,my_dates)

In [237]:
sf_refined

,date,text,geocode,user_location
0,2020-10-26 19:00:21,"More than 91,000 PG&amp;E customers in every B...",None,"San Francisco, CA"
1,2020-10-26 18:54:50,Planned power outages affect hundreds of thous...,None,"Oakland, CA"
2,2020-10-26 18:47:01,Mayor Jesse Arreguin spoke with @KTVU's Heathe...,None,"Berkeley, CA"
3,2020-10-26 18:35:03,A recent article in @sciam on #climatechange a...,None,"Berkeley, California"
4,2020-10-26 18:24:54,The wildfires are a reminder of an unpleasant ...,None,"Oakland, CA"
...,...,...,...,...
763,2020-10-18 00:12:30,Northern CA is home now but I grew up in North...,None,"San Francisco, CA"
764,2020-10-17 21:17:59,@polarscribe LN Curtis &amp; Sons has location...,None,"Oakland, CA"
765,2020-10-17 20:13:13,"Through October 2020, we are donating $100 to ...",None,"San Francisco, CA"
766,2020-10-17 19:01:03,The California wildfires this year have been d...,None,"Oakland, CA"


In [238]:
refined_tweets = pd.concat([sd_tweets,la_refined,santa_rosa,sf_refined])

In [240]:
refined_tweets.reset_index(inplace=True,drop=True)

In [242]:
refined_tweets.to_csv('twitter/refined_tweets.csv',index=False)

In [243]:
refined_tweets

,date,text,geocode,user_location
0,2020-10-26 17:58:23,Hundreds of thousands lose power as Northern C...,None,San Diego
1,2020-10-26 14:38:19,Two Colorado Wildfires Turn Deadly As Thousand...,None,"Chula Vista, CA"
2,2020-10-26 13:21:50,Large swaths of California were without power ...,None,"San Diego, CA"
3,2020-10-26 08:24:47,@Hey_its_Patti @SVNewsAlerts And the wildfires...,None,"San Diego, CA"
4,2020-10-26 03:50:02,With #California bracing for #FireSeason PG&am...,None,"San Diego, California"
...,...,...,...,...
2267,2020-10-18 00:12:30,Northern CA is home now but I grew up in North...,None,"San Francisco, CA"
2268,2020-10-17 21:17:59,@polarscribe LN Curtis &amp; Sons has location...,None,"Oakland, CA"
2269,2020-10-17 20:13:13,"Through October 2020, we are donating $100 to ...",None,"San Francisco, CA"
2270,2020-10-17 19:01:03,The California wildfires this year have been d...,None,"Oakland, CA"


In [253]:
bakersfield_geo = '35.36723,-119.01886,50mi'

In [254]:
bakersfield = get_tweets(bakersfield_geo,my_dates)

In [255]:
bakersfield

,date,text,geocode,user_location
0,2020-10-26 04:56:33,Hundreds of thousands lose power as Northern C...,None,Bakersfield
1,2020-10-25 05:04:50,Bakersfield is the only city I know where ther...,Place(_api=<tweepy.api.API object at 0x0000023...,"Bakersfield, CA"
2,2020-10-24 20:06:13,@markgrossi No restrictions until November 1! ...,None,"Kern County, California"
3,2020-10-24 02:55:25,"PG&amp;E said that 649 Kern County customers, ...",None,"Bakersfield, CA"
4,2020-10-23 05:42:01,Colorado wildfires have forced the closure of ...,None,Bakersfield
...,...,...,...,...
57,2020-10-17 22:41:12,Trump administration rejects California's disa...,None,"Bakersfield, CA"
58,2020-10-19 20:19:41,@terri_____ Wildfires,None,"Weldon, California"
59,2020-10-18 16:51:41,@catturd2 Im in Commifornia.... we arent getti...,None,"Bakersfield, CA"
60,2020-10-18 00:33:58,"@BlueSteelDC When wildfires are bad, LA looks ...",None,"Kern County, California"


In [256]:
mariposa_geo = '37.58147,-119.90545,75mi'

In [257]:
mariposa = get_tweets(mariposa_geo,my_dates)

In [258]:
mariposa

,date,text,geocode,user_location
0,2020-10-26 19:25:03,PUBLIC SAFETY POWER SHUTOFFS: Large swaths of ...,None,"North Coast, CA"
1,2020-10-26 18:50:52,@NBCNews So how come they haven't upgrade thei...,None,"Outer Space, CA"
2,2020-10-26 18:31:31,1 Million Could Lose Power as Negligent Commun...,None,"Fresno, CA"
3,2020-10-26 18:30:39,#SilveradoFire: Tens of thousands of residents...,None,Southern California
4,2020-10-26 18:21:01,"On top of everything else, I’m distracted toda...",None,Southern California
...,...,...,...,...
719,2020-10-17 21:55:00,"Trump administration rejects, then approves, e...",None,"NoHo Arts District, CA"
720,2020-10-17 19:48:19,"California Braces for Another Day of Heat, Win...",None,"Stockton, Ca"
721,2020-10-17 19:47:08,Aerial view of Napa Valley wineries damaged by...,None,"Stockton, Ca"
722,2020-10-17 18:45:04,Nearly two months after the SQF Complex Fire s...,None,"Fresno, CA"


In [259]:
refined_tweets = pd.concat([refined_tweets,bakersfield,mariposa])

In [261]:
refined_tweets.reset_index(inplace=True,drop=True)

In [263]:
refined_tweets.to_csv('twitter/refined_tweets.csv',index=False)

In [264]:
 refined_tweets

,date,text,geocode,user_location
0,2020-10-26 17:58:23,Hundreds of thousands lose power as Northern C...,None,San Diego
1,2020-10-26 14:38:19,Two Colorado Wildfires Turn Deadly As Thousand...,None,"Chula Vista, CA"
2,2020-10-26 13:21:50,Large swaths of California were without power ...,None,"San Diego, CA"
3,2020-10-26 08:24:47,@Hey_its_Patti @SVNewsAlerts And the wildfires...,None,"San Diego, CA"
4,2020-10-26 03:50:02,With #California bracing for #FireSeason PG&am...,None,"San Diego, California"
...,...,...,...,...
3053,2020-10-17 21:55:00,"Trump administration rejects, then approves, e...",None,"NoHo Arts District, CA"
3054,2020-10-17 19:48:19,"California Braces for Another Day of Heat, Win...",None,"Stockton, Ca"
3055,2020-10-17 19:47:08,Aerial view of Napa Valley wineries damaged by...,None,"Stockton, Ca"
3056,2020-10-17 18:45:04,Nearly two months after the SQF Complex Fire s...,None,"Fresno, CA"


In [265]:
redding_geo = '40.5876,-122.39255,100mi'

In [266]:
redding = get_tweets(redding_geo,my_dates)

In [267]:
redding

,date,text,geocode,user_location
0,2020-10-26 19:24:08,SAN FRANCISCO (AP) — Large swaths of Californi...,None,"Eureka, CA"
1,2020-10-26 03:57:23,Winds are starting to pick up in Potter Valley...,None,"Mendocino County, California"
2,2020-10-25 22:30:41,"California wildfires: High winds, record dry c...",None,"Susanville, CA"
3,2020-10-25 21:20:31,At least four wildfires break out in Shasta Co...,None,"Redding, California"
4,2020-10-25 02:00:00,The strongest wind event of the season arrives...,None,"Chico, CA"
...,...,...,...,...
124,2020-10-18 20:10:00,LATEST on California wildfires. Are you stayin...,None,"Chico, CA"
125,2020-10-18 18:06:44,Highlights: \n-Hiking in Upper Bidwell Park wh...,None,"Chico, CA"
126,2020-10-18 08:50:26,By removing dead and damaged trees and compac...,None,"Arcata, CA"
127,2020-10-18 02:30:00,LATEST on California wildfires. https://t.co/w...,None,"Chico, CA"


In [270]:
sacramento_geo = '38.57933,-121.4909,30mi'

In [271]:
sacramento = get_tweets(sacramento_geo,my_dates)

In [272]:
sacramento

,date,text,geocode,user_location
0,2020-10-26 18:48:26,A #disaster could happen at any moment like #w...,None,"Sacramento, CA"
1,2020-10-26 18:15:09,It may be many months before the full personal...,None,Sacramento
2,2020-10-26 17:45:27,California Wildfires Advisory Information for ...,None,"Headquarters Sacramento, CA"
3,2020-10-26 17:21:55,Conditions in California are currently high-ri...,None,"Sacramento, CA"
4,2020-10-26 17:05:53,Power Cut to Hundreds of Thousands in Californ...,Place(_api=<tweepy.api.API object at 0x0000023...,"Vacaville, CA"
...,...,...,...,...
680,2020-10-18 02:04:16,Firefighters will tell you this is SUPER rare....,None,"Sacramento, CA"
681,2020-10-18 00:00:01,California continues to experience record wild...,None,"Sacramento, CA"
682,2020-10-17 21:43:09,"Louisiana Hurricane Delta hit 10/6, Gov reques...",None,"Folsom, Ca"
683,2020-10-17 21:34:00,"@laurahelmuth @AliVelshi @MSNBC And of course,...",None,"Sacramento, CA"


In [273]:
refined_tweets = pd.concat([refined_tweets,redding,sacramento])

In [275]:
refined_tweets.reset_index(inplace=True,drop=True)

In [276]:
refined_tweets

,date,text,geocode,user_location
0,2020-10-26 17:58:23,Hundreds of thousands lose power as Northern C...,None,San Diego
1,2020-10-26 14:38:19,Two Colorado Wildfires Turn Deadly As Thousand...,None,"Chula Vista, CA"
2,2020-10-26 13:21:50,Large swaths of California were without power ...,None,"San Diego, CA"
3,2020-10-26 08:24:47,@Hey_its_Patti @SVNewsAlerts And the wildfires...,None,"San Diego, CA"
4,2020-10-26 03:50:02,With #California bracing for #FireSeason PG&am...,None,"San Diego, California"
...,...,...,...,...
3867,2020-10-18 02:04:16,Firefighters will tell you this is SUPER rare....,None,"Sacramento, CA"
3868,2020-10-18 00:00:01,California continues to experience record wild...,None,"Sacramento, CA"
3869,2020-10-17 21:43:09,"Louisiana Hurricane Delta hit 10/6, Gov reques...",None,"Folsom, Ca"
3870,2020-10-17 21:34:00,"@laurahelmuth @AliVelshi @MSNBC And of course,...",None,"Sacramento, CA"


In [277]:
refined_tweets.to_csv('twitter/best_tweets_updated.csv',index=False)

In [7]:
geolocator = Nominatim(user_agent="user",timeout=1)
locator = geolocator.geocode('Napa, CA')

In [11]:
locator[1]

(43.2290107, -79.9755399)

In [23]:
napa_geo = '38.29724,-122.289406,20mi'

In [24]:
napa = get_tweets(napa_geo,my_dates)

In [25]:
napa

,date,text,geocode,user_location
0,2020-10-26 19:30:32,Want to create wildfire resilient and environm...,None,Sonoma State University
1,2020-10-26 18:36:39,Phew! No major #wildfires so far in the North ...,None,"Napa, CA"
2,2020-10-26 18:06:06,The state cut power to hundreds of thousands t...,None,"Napa, California"
3,2020-10-26 17:05:53,Power Cut to Hundreds of Thousands in Californ...,Place(_api=<tweepy.api.API object at 0x0000016...,"Vacaville, CA"
4,2020-10-26 14:57:05,Winds exceeding 80 mph were recorded at the to...,None,"Napa, California"
...,...,...,...,...
215,2020-10-19 02:56:06,@terri_____ Wine country and wildfires.,None,"Rohnert Park, CA"
216,2020-10-18 01:33:04,While the recent wildfires in Western states h...,None,"Napa, California"
217,2020-10-17 21:03:05,Napa County is still recovering from the Octob...,None,"Napa, California"
218,2020-10-18 01:33:04,While the recent wildfires in Western states h...,None,"Napa, California"


In [20]:
eureka_geo = '40.80327,-124.15953,100mi'

In [21]:
eureka = get_tweets(eureka_geo,my_dates)

In [22]:
eureka

,date,text,geocode,user_location
0,2020-10-26 19:24:08,SAN FRANCISCO (AP) — Large swaths of Californi...,None,"Eureka, CA"
1,2020-10-25 21:20:31,At least four wildfires break out in Shasta Co...,None,"Redding, California"
2,2020-10-22 17:07:03,Red Cross is providing funds of $450 to househ...,None,"Eureka, CA"
3,2020-10-21 19:18:18,TONIGHT! Join @lenyaqd alongside other thought...,None,"Hayfork, CA"
4,2020-10-21 19:10:45,"@DBrozeLiveFree The ""impact of wildfires"" vide...",None,"Ft. Bragg, California"
5,2020-10-21 16:50:06,The Red Cross is providing Immediate Assistanc...,None,"Eureka, CA"
6,2020-10-19 03:10:48,@realDonaldTrump Nancy and having aren’t the o...,None,"Anderson, CA"
7,2020-10-18 08:50:26,By removing dead and damaged trees and compac...,None,"Arcata, CA"
8,2020-10-18 01:26:47,"Evening Update for KNF Wildfires, October 17, ...",None,"Yreka, CA"
9,2020-10-25 21:20:31,At least four wildfires break out in Shasta Co...,None,"Redding, California"


In [6]:
refined_tweets = pd.read_csv('twitter/best_tweets_updated.csv')

In [26]:
refined_tweets = pd.concat([refined_tweets,napa,eureka])

In [28]:
refined_tweets.reset_index(inplace=True,drop=True)

In [30]:
refined_tweets.to_csv('twitter/best_tweets_updated.csv')

In [31]:
refined_tweets

,date,text,geocode,user_location
0,2020-10-26 17:58:23,Hundreds of thousands lose power as Northern C...,NaN,San Diego
1,2020-10-26 14:38:19,Two Colorado Wildfires Turn Deadly As Thousand...,NaN,"Chula Vista, CA"
2,2020-10-26 13:21:50,Large swaths of California were without power ...,NaN,"San Diego, CA"
3,2020-10-26 08:24:47,@Hey_its_Patti @SVNewsAlerts And the wildfires...,NaN,"San Diego, CA"
4,2020-10-26 03:50:02,With #California bracing for #FireSeason PG&am...,NaN,"San Diego, California"
...,...,...,...,...
4139,2020-10-19 03:10:48,@realDonaldTrump Nancy and having aren’t the o...,None,"Anderson, CA"
4140,2020-10-18 08:50:26,By removing dead and damaged trees and compac...,None,"Arcata, CA"
4141,2020-10-18 01:26:47,"Evening Update for KNF Wildfires, October 17, ...",None,"Yreka, CA"
4142,2020-10-18 08:50:26,By removing dead and damaged trees and compac...,None,"Arcata, CA"


In [33]:
import re

In [34]:
def remove_url(text):
    url = re.compile("https?://\S+|www\.\S+")
    return url.sub(r'',text)

In [35]:
refined_tweets['text'] = refined_tweets['text'].apply(lambda x: remove_url(x))

In [37]:
refined_tweets.tail()

,date,text,geocode,user_location
4139,2020-10-19 03:10:48,@realDonaldTrump Nancy and having aren’t the o...,None,"Anderson, CA"
4140,2020-10-18 08:50:26,By removing dead and damaged trees and compac...,None,"Arcata, CA"
4141,2020-10-18 01:26:47,"Evening Update for KNF Wildfires, October 17, ...",None,"Yreka, CA"
4142,2020-10-18 08:50:26,By removing dead and damaged trees and compac...,None,"Arcata, CA"
4143,2020-10-18 01:26:47,"Evening Update for KNF Wildfires, October 17, ...",None,"Yreka, CA"


In [38]:
def create_mentions(text):
    url = re.findall("@\S+",text)
    return url

In [41]:
def remove_mentions(text):
    url = re.compile("@\S+")
    return url.sub(r"",text)

In [39]:
refined_tweets['mentions'] = refined_tweets['text'].apply(lambda x: create_mentions(x))

In [42]:
refined_tweets['text'] = refined_tweets['text'].apply(lambda x: remove_mentions(x))

In [43]:
refined_tweets

,date,text,geocode,user_location,mentions
0,2020-10-26 17:58:23,Hundreds of thousands lose power as Northern C...,NaN,San Diego,[]
1,2020-10-26 14:38:19,Two Colorado Wildfires Turn Deadly As Thousand...,NaN,"Chula Vista, CA",[@YouTube]
2,2020-10-26 13:21:50,Large swaths of California were without power ...,NaN,"San Diego, CA",[]
3,2020-10-26 08:24:47,"And the wildfires, too. should now be consi...",NaN,"San Diego, CA","[@Hey_its_Patti, @SVNewsAlerts, @FBI]"
4,2020-10-26 03:50:02,With #California bracing for #FireSeason PG&am...,NaN,"San Diego, California",[]
...,...,...,...,...,...
4139,2020-10-19 03:10:48,Nancy and having aren’t the ones who denied C...,None,"Anderson, CA",[@realDonaldTrump]
4140,2020-10-18 08:50:26,By removing dead and damaged trees and compac...,None,"Arcata, CA",[]
4141,2020-10-18 01:26:47,"Evening Update for KNF Wildfires, October 17, ...",None,"Yreka, CA",[]
4142,2020-10-18 08:50:26,By removing dead and damaged trees and compac...,None,"Arcata, CA",[]


In [44]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\seanp\AppData\Roaming\nltk_data...


True

In [45]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [46]:
sid = SentimentIntensityAnalyzer()

In [47]:
refined_tweets['scores'] = refined_tweets['text'].apply(lambda tweet: sid.polarity_scores(tweet))

In [48]:
refined_tweets['compound'] = refined_tweets['scores'].apply()

,date,text,geocode,user_location,mentions,scores
0,2020-10-26 17:58:23,Hundreds of thousands lose power as Northern C...,NaN,San Diego,[],"{'neg': 0.16, 'neu': 0.752, 'pos': 0.088, 'com..."
1,2020-10-26 14:38:19,Two Colorado Wildfires Turn Deadly As Thousand...,NaN,"Chula Vista, CA",[@YouTube],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,2020-10-26 13:21:50,Large swaths of California were without power ...,NaN,"San Diego, CA",[],"{'neg': 0.077, 'neu': 0.771, 'pos': 0.152, 'co..."
3,2020-10-26 08:24:47,"And the wildfires, too. should now be consi...",NaN,"San Diego, CA","[@Hey_its_Patti, @SVNewsAlerts, @FBI]","{'neg': 0.199, 'neu': 0.642, 'pos': 0.159, 'co..."
4,2020-10-26 03:50:02,With #California bracing for #FireSeason PG&am...,NaN,"San Diego, California",[],"{'neg': 0.047, 'neu': 0.847, 'pos': 0.106, 'co..."
...,...,...,...,...,...,...
4139,2020-10-19 03:10:48,Nancy and having aren’t the ones who denied C...,None,"Anderson, CA",[@realDonaldTrump],"{'neg': 0.166, 'neu': 0.753, 'pos': 0.081, 'co..."
4140,2020-10-18 08:50:26,By removing dead and damaged trees and compac...,None,"Arcata, CA",[],"{'neg': 0.218, 'neu': 0.667, 'pos': 0.115, 'co..."
4141,2020-10-18 01:26:47,"Evening Update for KNF Wildfires, October 17, ...",None,"Yreka, CA",[],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
4142,2020-10-18 08:50:26,By removing dead and damaged trees and compac...,None,"Arcata, CA",[],"{'neg': 0.218, 'neu': 0.667, 'pos': 0.115, 'co..."
